# Đặt Phòng Khách Sạn với Middleware Thành Viên Ưu Tiên

Notebook này minh họa **middleware dựa trên hàm** sử dụng Microsoft Agent Framework. Chúng ta sẽ phát triển từ ví dụ về luồng công việc có điều kiện bằng cách thêm một lớp middleware cho phép các thành viên ưu tiên hưởng đặc quyền đặc biệt.

## Những Điều Bạn Sẽ Học:
1. **Middleware Dựa trên Hàm**: Can thiệp và chỉnh sửa kết quả của hàm
2. **Truy Cập Ngữ Cảnh**: Đọc và chỉnh sửa `context.result` sau khi thực thi
3. **Triển Khai Logic Kinh Doanh**: Lợi ích cho thành viên ưu tiên
4. **Ghi Đè Kết Quả**: Thay đổi kết quả của hàm dựa trên trạng thái người dùng
5. **Cùng Một Luồng Công Việc, Kết Quả Khác Nhau**: Thay đổi hành vi dựa trên middleware

## Kiến Trúc Luồng Công Việc với Middleware:

```
User Input: "I want to book a hotel in Paris"
                    ↓
        [availability_agent]
        - Calls hotel_booking tool
        - 🌟 priority_check middleware intercepts
        - Checks user membership status
        - IF priority + no rooms → Override to available!
        - Returns BookingCheckResult
                    ↓
        Conditional Routing
           /                    \
    [has_availability]    [no_availability]
          ↓                      ↓
    [booking_agent]        [alternative_agent]
    (Priority override!)   (Regular users)
          ↓                      ↓
       [display_result executor]
```

## Điểm Khác Biệt Chính so với Luồng Công Việc Có Điều Kiện:

**Không Có Middleware** (14-conditional-workflow.ipynb):
- Paris không có phòng → Chuyển đến alternative_agent

**Có Middleware** (notebook này):
- Người dùng thường + Paris → Không có phòng → Chuyển đến alternative_agent
- Thành viên ưu tiên + Paris → 🌟 Middleware ghi đè! → Có phòng → Chuyển đến booking_agent

## Yêu Cầu:
- Đã cài đặt Microsoft Agent Framework
- Hiểu về luồng công việc có điều kiện (xem 14-conditional-workflow.ipynb)
- Token GitHub hoặc khóa API OpenAI
- Hiểu cơ bản về mẫu middleware


In [2]:
import asyncio
import json
import os
from collections.abc import Awaitable, Callable
from typing import Annotated, Any, Never

from agent_framework import (
    AgentExecutor,
    AgentExecutorRequest,
    AgentExecutorResponse,
    ChatMessage,
    FunctionInvocationContext,
    Role,
    WorkflowBuilder,
    WorkflowContext,
    ai_function,
    executor,
)

# 🤖 GitHub Models or OpenAI client integration
from agent_framework.openai import OpenAIChatClient
from dotenv import load_dotenv
from IPython.display import HTML, display
from pydantic import BaseModel

print("✅ All imports successful!")

✅ All imports successful!


## Bước 1: Định nghĩa các Mô hình Pydantic cho Kết quả Có cấu trúc

Các mô hình này định nghĩa **schema** mà các agent sẽ trả về. Chúng tôi đã thêm trường `priority_override` để theo dõi khi middleware sửa đổi kết quả khả dụng.


In [3]:
class BookingCheckResult(BaseModel):
    """Result from checking hotel availability at a destination."""

    destination: str
    has_availability: bool
    message: str
    priority_override: bool = False  # 🆕 NEW! Tracks if middleware overrode the result


class AlternativeResult(BaseModel):
    """Suggested alternative destination when no rooms available."""

    alternative_destination: str
    reason: str


class BookingConfirmation(BaseModel):
    """Booking suggestion when rooms are available."""

    destination: str
    action: str
    message: str


print("✅ Pydantic models defined:")
print("   - BookingCheckResult (availability check with priority_override)")
print("   - AlternativeResult (alternative suggestion)")
print("   - BookingConfirmation (booking confirmation)")

✅ Pydantic models defined:
   - BookingCheckResult (availability check with priority_override)
   - AlternativeResult (alternative suggestion)
   - BookingConfirmation (booking confirmation)


## Bước 2: Định nghĩa Cơ sở Dữ liệu Thành viên Ưu tiên

Trong bản demo này, chúng ta sẽ mô phỏng một cơ sở dữ liệu thành viên ưu tiên. Trong môi trường sản xuất, điều này sẽ truy vấn một cơ sở dữ liệu thực hoặc API.

**Thành viên Ưu tiên:**
- `alice@example.com` - Thành viên VIP
- `bob@example.com` - Thành viên Premium  
- `priority_user` - Tài khoản thử nghiệm


In [4]:
# Simulated priority members database
PRIORITY_MEMBERS = {
    "alice@example.com",
    "bob@example.com",
    "priority_user",
}

# Global variable to track current user (in real app, use proper session management)
current_user_id = "regular_user"  # Default: regular user


def set_user(user_id: str):
    """Set the current user for the session."""
    global current_user_id
    current_user_id = user_id
    is_priority = user_id in PRIORITY_MEMBERS
    status = "🌟 PRIORITY MEMBER" if is_priority else "👤 Regular User"

    display(
        HTML(f"""
        <div style='padding: 15px; background: {"linear-gradient(135deg, #FFD700 0%, #FFA500 100%)" if is_priority else "#e3f2fd"}; 
                    border-left: 4px solid {"#FF6B35" if is_priority else "#2196f3"}; border-radius: 4px; margin: 10px 0;'>
            <strong>👤 Current User Set:</strong> {user_id}<br>
            <strong>Status:</strong> {status}
        </div>
    """)
    )


print("✅ Priority members database created")
print(f"   Priority members: {len(PRIORITY_MEMBERS)} users")

✅ Priority members database created
   Priority members: 3 users


## Bước 3: Tạo Công Cụ Đặt Phòng Khách Sạn

Giống như quy trình làm việc có điều kiện, nhưng bây giờ nó sẽ được chặn bởi middleware!


In [5]:
@ai_function(description="Check hotel room availability for a destination city")
def hotel_booking(destination: Annotated[str, "The destination city to check for hotel rooms"]) -> str:
    """
    Simulates checking hotel room availability.
    
    Returns JSON string with availability status.
    """
    display(
        HTML(f"""
        <div style='padding: 15px; background: #e3f2fd; border-left: 4px solid #2196f3; border-radius: 4px; margin: 10px 0;'>
            <strong>🔍 Tool Invoked:</strong> hotel_booking("{destination}")
        </div>
    """)
    )

    # Simulate availability check
    cities_with_rooms = ["stockholm", "seattle", "tokyo", "london", "amsterdam"]
    has_rooms = destination.lower() in cities_with_rooms

    result = {"has_availability": has_rooms, "destination": destination}

    return json.dumps(result)


print("✅ hotel_booking tool created with @ai_function decorator")

✅ hotel_booking tool created with @ai_function decorator


## Bước 4: 🌟 Tạo Middleware Kiểm Tra Ưu Tiên (TÍNH NĂNG CHÍNH!)

Đây là **chức năng cốt lõi** của notebook này. Middleware:

1. **Chặn** lời gọi hàm hotel_booking
2. **Thực thi** hàm như bình thường bằng cách gọi `next(context)`
3. **Kiểm tra** kết quả trong `context.result`
4. **Ghi đè** kết quả nếu người dùng là ưu tiên và không có phòng trống
5. **Trả lại** kết quả đã chỉnh sửa cho agent

**Mẫu chính:**
```python
async def my_middleware(context, next):
    await next(context)  # Execute function
    # Now context.result contains the function's output
    if some_condition:
        context.result = new_value  # Override!
```


In [6]:
async def priority_check_middleware(
    context: FunctionInvocationContext,
    next: Callable[[FunctionInvocationContext], Awaitable[None]],
) -> None:
    """
    Function middleware that overrides hotel_booking results for priority members.
    
    Workflow:
    1. Let the function execute normally
    2. Check if user is a priority member
    3. If priority + no availability → Override to make rooms available!
    4. Agent will then route to booking path instead of alternative path
    """
    function_name = context.function.name

    display(
        HTML(f"""
        <div style='padding: 12px; background: #fff3e0; border-left: 4px solid #ff9800; border-radius: 4px; margin: 10px 0;'>
            <strong>🔄 Middleware:</strong> Intercepting {function_name}...
        </div>
    """)
    )

    # Execute the original function
    await next(context)

    # Now inspect and potentially modify the result
    if context.result and function_name == "hotel_booking":
        result_data = json.loads(context.result)
        destination = result_data.get("destination", "")
        has_availability = result_data.get("has_availability", False)

        # Check if user is priority member
        is_priority = current_user_id in PRIORITY_MEMBERS

        # Override logic: Priority member + no availability → Make available!
        if is_priority and not has_availability:
            display(
                HTML(f"""
                <div style='padding: 20px; background: linear-gradient(135deg, #FFD700 0%, #FFA500 100%); 
                            border-radius: 8px; margin: 10px 0; box-shadow: 0 4px 12px rgba(255,165,0,0.4);'>
                    <h3 style='margin: 0 0 10px 0; color: #333;'>🌟 PRIORITY OVERRIDE ACTIVATED! 🌟</h3>
                    <p style='margin: 0; color: #555; line-height: 1.6;'>
                        <strong>User:</strong> {current_user_id}<br>
                        <strong>Status:</strong> VIP Priority Member<br>
                        <strong>Action:</strong> Overriding "No Availability" for {destination}<br>
                        <strong>Result:</strong> ✅ Rooms now available for priority booking!
                    </p>
                </div>
            """)
            )

            # Override the result!
            result_data["has_availability"] = True
            result_data["priority_override"] = True
            context.result = json.dumps(result_data)

        elif not has_availability:
            display(
                HTML(f"""
                <div style='padding: 12px; background: #ffebee; border-left: 4px solid #f44336; border-radius: 4px; margin: 10px 0;'>
                    <strong>ℹ️ Middleware:</strong> No priority override (user: {current_user_id})
                </div>
            """)
            )


print("✅ priority_check_middleware created")
print("   - Intercepts hotel_booking function")
print("   - Overrides availability for priority members")

✅ priority_check_middleware created
   - Intercepts hotel_booking function
   - Overrides availability for priority members


## Bước 5: Định nghĩa các hàm điều kiện cho định tuyến

Các hàm điều kiện giống như trong quy trình làm việc có điều kiện - chúng kiểm tra đầu ra có cấu trúc để xác định định tuyến.


In [7]:
def has_availability_condition(message: Any) -> bool:
    """Condition for routing when hotels ARE available (including priority overrides!)."""
    if not isinstance(message, AgentExecutorResponse):
        return True

    try:
        result = BookingCheckResult.model_validate_json(message.agent_run_response.text)

        # Check if this was a priority override
        override_indicator = " 🌟" if result.priority_override else ""

        display(
            HTML(f"""
            <div style='padding: 12px; background: #c8e6c9; border-left: 4px solid #4caf50; border-radius: 4px; margin: 10px 0;'>
                <strong>✅ Condition Check:</strong> has_availability = <strong>{result.has_availability}</strong> for {result.destination}{override_indicator}
            </div>
        """)
        )

        return result.has_availability
    except Exception as e:
        display(
            HTML(f"""
            <div style='padding: 12px; background: #ffcdd2; border-left: 4px solid #f44336; border-radius: 4px; margin: 10px 0;'>
                <strong>⚠️  Error:</strong> {str(e)}
            </div>
        """)
        )
        return False


def no_availability_condition(message: Any) -> bool:
    """Condition for routing when hotels are NOT available."""
    if not isinstance(message, AgentExecutorResponse):
        return False

    try:
        result = BookingCheckResult.model_validate_json(message.agent_run_response.text)

        display(
            HTML(f"""
            <div style='padding: 12px; background: #ffecb3; border-left: 4px solid #ff9800; border-radius: 4px; margin: 10px 0;'>
                <strong>❌ Condition Check:</strong> no_availability for {result.destination}
            </div>
        """)
        )

        return not result.has_availability
    except Exception:
        return False


print("✅ Condition functions defined")

✅ Condition functions defined


## Bước 6: Tạo Bộ Thực Thi Hiển Thị Tùy Chỉnh

Bộ thực thi giống như trước - hiển thị kết quả cuối cùng của quy trình làm việc.


In [8]:
@executor(id="display_result")
async def display_result(response: AgentExecutorResponse, ctx: WorkflowContext[Never, str]) -> None:
    """Display the final result as workflow output."""
    display(
        HTML("""
        <div style='padding: 15px; background: #f3e5f5; border-left: 4px solid #9c27b0; border-radius: 4px; margin: 10px 0;'>
            <strong>📤 Display Executor:</strong> Yielding workflow output
        </div>
    """)
    )

    await ctx.yield_output(response.agent_run_response.text)


print("✅ display_result executor created")

✅ display_result executor created


## Bước 7: Tải các biến môi trường

Cấu hình client LLM (GitHub Models hoặc OpenAI).


In [10]:
# Load environment variables
load_dotenv()

# Check for GitHub Models or OpenAI
chat_client = OpenAIChatClient(base_url=os.environ.get(
    "GITHUB_ENDPOINT"), api_key=os.environ.get("GITHUB_TOKEN"), model_id="gpt-4o")


## Bước 8: Tạo các tác nhân AI với Middleware

**ĐIỂM KHÁC BIỆT CHÍNH:** Khi tạo availability_agent, chúng ta truyền tham số `middleware`!

Đây là cách chúng ta đưa priority_check_middleware vào quy trình gọi hàm của tác nhân.


In [ ]:
# Agent 1: Check availability with tool + middleware
availability_agent = AgentExecutor(
    chat_client.create_agent(
        instructions=(
            "You are a hotel booking assistant that checks room availability. "
            "Use the hotel_booking tool to check if rooms are available at the destination. "
            "Return JSON with fields: destination (string), has_availability (bool), message (string), "
            "and priority_override (bool, true if priority member got special access). "
            "The message should summarize the availability status and mention if priority override occurred."
        ),
        tools=[hotel_booking],
        response_format=BookingCheckResult,
        middleware=[priority_check_middleware],  # 🌟 MIDDLEWARE INJECTION!
    ),
    id="availability_agent",
)

# Agent 2: Suggest alternative (when no rooms)
alternative_agent = AgentExecutor(
    chat_client.create_agent(
        instructions=(
            "You are a helpful travel assistant. When a user cannot find hotels in their requested city, "
            "suggest an alternative nearby city that has availability. "
            "Return JSON with fields: alternative_destination (string) and reason (string). "
            "Make your suggestion sound appealing and helpful."
        ),
        response_format=AlternativeResult,
    ),
    id="alternative_agent",
)

# Agent 3: Suggest booking (when rooms available)
booking_agent = AgentExecutor(
    chat_client.create_agent(
        instructions=(
            "You are a booking assistant. The user has found available hotel rooms. "
            "Encourage them to book by highlighting the destination's appeal. "
            "If priority_override is true in the input, mention that they received priority member access. "
            "Return JSON with fields: destination (string), action (string), and message (string). "
            "The action should be 'book_now' and message should be encouraging."
        ),
        response_format=BookingConfirmation,
    ),
    id="booking_agent",
)

display(
    HTML("""
    <div style='padding: 15px; background: #e3f2fd; border-left: 4px solid #2196f3; border-radius: 4px; margin: 10px 0;'>
        <strong>✅ Created 3 Agents:</strong>
        <ul style='margin: 10px 0 0 0;'>
            <li><strong>availability_agent</strong> - WITH priority_check_middleware 🌟</li>
            <li><strong>alternative_agent</strong> - Suggests alternative cities</li>
            <li><strong>booking_agent</strong> - Encourages booking</li>
        </ul>
    </div>
""")
)

## Bước 9: Xây dựng Quy trình Làm việc

Cấu trúc quy trình làm việc giống như trước - định tuyến có điều kiện dựa trên khả dụng.


In [12]:
# Build the workflow with conditional routing
workflow = (
    WorkflowBuilder()
    .set_start_executor(availability_agent)
    # NO AVAILABILITY PATH
    .add_edge(availability_agent, alternative_agent, condition=no_availability_condition)
    .add_edge(alternative_agent, display_result)
    # HAS AVAILABILITY PATH (can be triggered by middleware override!)
    .add_edge(availability_agent, booking_agent, condition=has_availability_condition)
    .add_edge(booking_agent, display_result)
    .build()
)

display(
    HTML("""
    <div style='padding: 20px; background: linear-gradient(135deg, #667eea 0%, #764ba2 100%); color: white; border-radius: 8px; margin: 10px 0;'>
        <h3 style='margin: 0 0 15px 0;'>✅ Workflow Built Successfully!</h3>
        <p style='margin: 0; line-height: 1.6;'>
            <strong>Conditional Routing (Middleware-Aware):</strong><br>
            • If <strong>NO availability</strong> → alternative_agent → display_result<br>
            • If <strong>availability</strong> (or 🌟 <strong>priority override</strong>) → booking_agent → display_result
        </p>
    </div>
""")
)

## Bước 10: Kiểm tra trường hợp 1 - Người dùng thông thường ở Paris (Không ghi đè)

Một người dùng thông thường cố gắng đặt phòng ở Paris → Không có phòng → Chuyển hướng đến alternative_agent


In [13]:
# Set as regular user
set_user("regular_user")

display(
    HTML("""
    <div style='padding: 20px; background: #fff3e0; border-left: 4px solid #ff9800; border-radius: 8px; margin: 20px 0;'>
        <h3 style='margin: 0 0 10px 0; color: #e65100;'>🧪 TEST CASE 1: Regular User + Paris</h3>
        <p style='margin: 0;'><strong>Expected:</strong> No rooms → No middleware override → Alternative suggestion</p>
    </div>
""")
)

# Create request
request_regular = AgentExecutorRequest(
    messages=[ChatMessage(Role.USER, text="I want to book a hotel in Paris")], should_respond=True
)

# Run workflow
events_regular = await workflow.run(request_regular)
outputs_regular = events_regular.get_outputs()

# Display results
if outputs_regular:
    result_regular = AlternativeResult.model_validate_json(outputs_regular[0])

    display(
        HTML(f"""
        <div style='padding: 25px; background: #fff; border: 2px solid #ff9800; border-radius: 12px; margin: 20px 0;'>
            <h3 style='margin: 0 0 15px 0; color: #e65100;'>📊 RESULT (Regular User)</h3>
            <div style='background: #fff3e0; padding: 20px; border-radius: 8px;'>
                <p style='margin: 0 0 10px 0;'><strong>Status:</strong> ❌ No rooms in Paris</p>
                <p style='margin: 0 0 10px 0;'><strong>Middleware:</strong> No priority override (regular user)</p>
                <p style='margin: 0 0 10px 0;'><strong>Alternative:</strong> 🏨 {result_regular.alternative_destination}</p>
                <p style='margin: 0;'><strong>Reason:</strong> {result_regular.reason}</p>
            </div>
        </div>
    """)
    )

## Bước 11: Trường hợp kiểm tra 2 - 🌟 Người dùng ưu tiên ở Paris (CÓ Ghi đè!)

Một thành viên ưu tiên cố gắng đặt phòng ở Paris → Ban đầu không có phòng → 🌟 Middleware ghi đè! → Chuyển đến booking_agent

**Đây là minh chứng quan trọng về sức mạnh của middleware!**


In [14]:
# Set as priority user
set_user("priority_user")

display(
    HTML("""
    <div style='padding: 20px; background: linear-gradient(135deg, #FFD700 0%, #FFA500 100%); border-radius: 8px; margin: 20px 0;'>
        <h3 style='margin: 0 0 10px 0; color: #333;'>🧪 TEST CASE 2: 🌟 Priority User + Paris</h3>
        <p style='margin: 0; color: #555;'><strong>Expected:</strong> No rooms → 🌟 MIDDLEWARE OVERRIDE → Rooms available → Booking suggestion!</p>
    </div>
""")
)

# Create request
request_priority = AgentExecutorRequest(
    messages=[ChatMessage(Role.USER, text="I want to book a hotel in Paris")], should_respond=True
)

# Run workflow
events_priority = await workflow.run(request_priority)
outputs_priority = events_priority.get_outputs()

# Display results
if outputs_priority:
    result_priority = BookingConfirmation.model_validate_json(outputs_priority[0])

    display(
        HTML(f"""
        <div style='padding: 25px; background: linear-gradient(135deg, #FFD700 0%, #FFA500 100%); border-radius: 12px; 
                    box-shadow: 0 8px 16px rgba(255,165,0,0.4); margin: 20px 0;'>
            <h3 style='margin: 0 0 15px 0; color: #333;'>🏆 RESULT (Priority Member) 🌟</h3>
            <div style='background: white; padding: 20px; border-radius: 8px;'>
                <p style='margin: 0 0 10px 0; font-size: 16px;'><strong>Status:</strong> ✅ Rooms Available (Priority Override!)</p>
                <p style='margin: 0 0 10px 0; font-size: 16px;'><strong>Middleware:</strong> 🌟 OVERRIDE ACTIVATED!</p>
                <p style='margin: 0 0 10px 0; font-size: 16px;'><strong>Destination:</strong> 🏨 {result_priority.destination}</p>
                <p style='margin: 0 0 10px 0; font-size: 16px;'><strong>Action:</strong> {result_priority.action}</p>
                <p style='margin: 0; font-size: 14px; color: #666;'><strong>Message:</strong> {result_priority.message}</p>
                <div style='margin-top: 15px; padding: 15px; background: #fff3cd; border-radius: 6px; border-left: 4px solid #FF6B35;'>
                    <strong>💡 What Just Happened:</strong><br>
                    1. hotel_booking tool returned "no availability"<br>
                    2. priority_check_middleware intercepted the result<br>
                    3. Middleware checked user status: priority_user ✅<br>
                    4. Middleware OVERRODE the result to "available"<br>
                    5. Workflow routed to booking_agent instead of alternative_agent!
                </div>
            </div>
        </div>
    """)
    )

## Bước 12: Kiểm tra trường hợp 3 - Người dùng ưu tiên ở Stockholm (Đã có sẵn)

Người dùng ưu tiên thử Stockholm → Phòng có sẵn → Không cần ghi đè → Chuyển đến booking_agent

Điều này cho thấy middleware chỉ hoạt động khi cần thiết!


In [15]:
# Priority user is still set from previous test

display(
    HTML("""
    <div style='padding: 20px; background: #e8f5e9; border-left: 4px solid #4caf50; border-radius: 8px; margin: 20px 0;'>
        <h3 style='margin: 0 0 10px 0; color: #1b5e20;'>🧪 TEST CASE 3: Priority User + Stockholm</h3>
        <p style='margin: 0;'><strong>Expected:</strong> Rooms available → No override needed → Booking suggestion</p>
    </div>
""")
)

# Create request
request_stockholm = AgentExecutorRequest(
    messages=[ChatMessage(Role.USER, text="I want to book a hotel in Stockholm")], should_respond=True
)

# Run workflow
events_stockholm = await workflow.run(request_stockholm)
outputs_stockholm = events_stockholm.get_outputs()

# Display results
if outputs_stockholm:
    result_stockholm = BookingConfirmation.model_validate_json(outputs_stockholm[0])

    display(
        HTML(f"""
        <div style='padding: 25px; background: linear-gradient(135deg, #4caf50 0%, #8bc34a 100%); color: white; border-radius: 12px; 
                    box-shadow: 0 4px 12px rgba(76,175,80,0.3); margin: 20px 0;'>
            <h3 style='margin: 0 0 15px 0;'>🏆 RESULT (Priority User - No Override Needed)</h3>
            <div style='background: white; color: #333; padding: 20px; border-radius: 8px;'>
                <p style='margin: 0 0 10px 0; font-size: 16px;'><strong>Status:</strong> ✅ Rooms Available (Natural)</p>
                <p style='margin: 0 0 10px 0; font-size: 16px;'><strong>Middleware:</strong> No override needed</p>
                <p style='margin: 0 0 10px 0; font-size: 16px;'><strong>Destination:</strong> 🏨 {result_stockholm.destination}</p>
                <p style='margin: 0 0 10px 0; font-size: 16px;'><strong>Action:</strong> {result_stockholm.action}</p>
                <p style='margin: 0; font-size: 14px; color: #666;'><strong>Message:</strong> {result_stockholm.message}</p>
                <div style='margin-top: 15px; padding: 15px; background: #e8f5e9; border-radius: 6px; border-left: 4px solid #4caf50;'>
                    <strong>💡 Middleware Behavior:</strong><br>
                    • hotel_booking returned "available" naturally<br>
                    • Middleware saw available = true → No override needed<br>
                    • Workflow proceeded normally to booking_agent
                </div>
            </div>
        </div>
    """)
    )

## Những Điểm Chính và Khái Niệm Middleware

### ✅ Những Gì Bạn Đã Học:

#### **1. Mẫu Middleware Dựa Trên Hàm**

Middleware chặn các lời gọi hàm bằng một hàm async đơn giản:

```python
async def my_middleware(
    context: FunctionInvocationContext,
    next: Callable,
) -> None:
    # Before function execution
    print("Intercepting...")
    
    # Execute the function
    await next(context)
    
    # After function execution - inspect result
    if context.result:
        # Modify result if needed
        context.result = modified_value
```

#### **2. Truy Cập Context và Ghi Đè Kết Quả**

- `context.function` - Truy cập hàm đang được gọi
- `context.arguments` - Đọc các tham số của hàm
- `context.kwargs` - Truy cập các tham số bổ sung
- `await next(context)` - Thực thi hàm
- `context.result` - Đọc/sửa đổi kết quả của hàm

#### **3. Triển Khai Logic Kinh Doanh**

Middleware của chúng ta triển khai lợi ích cho thành viên ưu tiên:
- **Người dùng thông thường**: Không thay đổi, quy trình tiêu chuẩn
- **Người dùng ưu tiên**: Ghi đè "không có sẵn" → "có sẵn"
- **Logic có điều kiện**: Chỉ ghi đè khi cần thiết

#### **4. Cùng Quy Trình, Kết Quả Khác Nhau**

Sức mạnh của middleware:
- ✅ Không thay đổi cấu trúc quy trình
- ✅ Không thay đổi hàm công cụ
- ✅ Không thay đổi logic định tuyến có điều kiện
- ✅ Chỉ cần middleware → Hành vi khác biệt!

### 🚀 Ứng Dụng Thực Tế:

1. **Tính Năng VIP/Premium**
   - Ghi đè giới hạn tốc độ cho người dùng premium
   - Cung cấp quyền truy cập ưu tiên vào tài nguyên
   - Mở khóa tính năng premium một cách linh hoạt

2. **A/B Testing**
   - Định tuyến người dùng đến các triển khai khác nhau
   - Thử nghiệm tính năng mới với người dùng cụ thể
   - Triển khai tính năng dần dần

3. **Bảo Mật & Tuân Thủ**
   - Kiểm tra các lời gọi hàm
   - Chặn các thao tác nhạy cảm
   - Thực thi các quy tắc kinh doanh

4. **Tối Ưu Hiệu Suất**
   - Lưu trữ kết quả cho người dùng cụ thể
   - Bỏ qua các thao tác tốn kém khi có thể
   - Phân bổ tài nguyên linh hoạt

5. **Xử Lý Lỗi & Thử Lại**
   - Bắt và xử lý lỗi một cách linh hoạt
   - Triển khai logic thử lại
   - Dự phòng sang các triển khai thay thế

6. **Ghi Log & Giám Sát**
   - Theo dõi thời gian thực thi hàm
   - Ghi log các tham số và kết quả
   - Giám sát các mẫu sử dụng

### 🔑 Sự Khác Biệt Chính So Với Decorators:

| Tính Năng | Decorator | Middleware |
|-----------|-----------|------------|
| **Phạm Vi** | Một hàm duy nhất | Tất cả các hàm trong agent |
| **Tính Linh Hoạt** | Cố định khi định nghĩa | Linh hoạt khi runtime |
| **Context** | Hạn chế | Toàn bộ context của agent |
| **Kết Hợp** | Nhiều decorators | Chuỗi middleware |
| **Nhận Biết Agent** | Không | Có (truy cập trạng thái agent) |

### 📚 Khi Nào Nên Sử Dụng Middleware:

✅ **Sử dụng middleware khi:**
- Bạn cần thay đổi hành vi dựa trên trạng thái người dùng/phiên
- Bạn muốn áp dụng logic cho nhiều hàm
- Bạn cần truy cập context ở cấp độ agent
- Bạn đang triển khai các mối quan tâm xuyên suốt (logging, auth, v.v.)

❌ **Không sử dụng middleware khi:**
- Xác thực đầu vào đơn giản (sử dụng Pydantic)
- Logic cụ thể cho từng hàm (giữ trong hàm)
- Thay đổi một lần (chỉ cần thay đổi hàm)

### 🎓 Mẫu Nâng Cao:

```python
# Multiple middleware (execution order matters!)
middleware=[
    logging_middleware,      # Logs first
    auth_middleware,         # Then checks auth
    cache_middleware,        # Then checks cache
    rate_limit_middleware,   # Then rate limits
    priority_check_middleware  # Finally priority check
]

# Conditional middleware execution
async def conditional_middleware(context, next):
    if should_execute(context):
        await next(context)
        # Modify result
    else:
        # Skip execution entirely
        context.result = cached_value
```

### 🔗 Các Khái Niệm Liên Quan:

- **Agent Middleware**: Chặn các lời gọi agent.run()
- **Function Middleware**: Chặn các lời gọi hàm công cụ (chúng ta đã sử dụng!)
- **Middleware Pipeline**: Chuỗi middleware thực thi theo thứ tự
- **Context Propagation**: Truyền trạng thái qua chuỗi middleware



---

**Tuyên bố miễn trừ trách nhiệm**:  
Tài liệu này đã được dịch bằng dịch vụ dịch thuật AI [Co-op Translator](https://github.com/Azure/co-op-translator). Mặc dù chúng tôi cố gắng đảm bảo độ chính xác, xin lưu ý rằng các bản dịch tự động có thể chứa lỗi hoặc không chính xác. Tài liệu gốc bằng ngôn ngữ bản địa nên được coi là nguồn thông tin chính thức. Đối với các thông tin quan trọng, nên sử dụng dịch vụ dịch thuật chuyên nghiệp của con người. Chúng tôi không chịu trách nhiệm về bất kỳ sự hiểu lầm hoặc diễn giải sai nào phát sinh từ việc sử dụng bản dịch này.
